# H&M Data Preprocessing and Cleaning with Polars

This notebook performs comprehensive preprocessing and cleaning of the H&M dataset using Polars for high-performance data processing, including:

- Duplicate removal
- Missing value handling (per user specifications)
- Outlier detection and handling
- Data validation
- Export to Parquet format

## Missing Value Handling Strategy

- **t_dat**: Interpolate based on customer patterns
- **customer_id/article_id**: Drop rows with missing IDs
- **price**: Fill with median price for that article_id
- **sales_channel_id**: Fill with mode
- **FN**: Fill with 0
- **Active**: Fill with "UNKNOWN"
- **club_member_status**: Fill with "INACTIVE"
- **fashion_news_frequency**: Fill with "NONE"
- **age**: Fill with median age
- **postal_code**: Fill with "UNKNOWN"
- **Categorical fields**: Fill with "UNKNOWN"
- **Numerical codes**: Fill with 0
- **detail_desc**: Fill with "NO_DESCRIPTION"


In [15]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configure Polars
pl.Config.set_streaming_chunk_size(100_000)

print(f"Polars version: {pl.__version__}")
print("Libraries imported successfully")

Polars version: 1.32.0
Libraries imported successfully


## 1. Data Loading


In [16]:
print("Loading raw data files...")

# Robust data loading with existence checks and clear error messages
def _safe_read_csv(path, **kwargs):
    if not Path(path).exists():
        raise FileNotFoundError(f"File not found: {path}\n"
                                "Please ensure the dataset is available at the specified path.")
    return pl.read_csv(path, **kwargs)

# Load transactions data
transactions_path = Path(data_path) / "transactions_train.csv"
print(f"Loading transactions from {transactions_path} ...")
transactions_df = _safe_read_csv(transactions_path, try_parse_dates=True)

# Load customers data
customers_path = Path(data_path) / "customers.csv"
print(f"Loading customers from {customers_path} ...")
customers_df = _safe_read_csv(customers_path, try_parse_dates=True)

# Load articles data
articles_path = Path(data_path) / "articles.csv"
print(f"Loading articles from {articles_path} ...")
articles_df = _safe_read_csv(articles_path, try_parse_dates=True)

print(f"Transactions: {transactions_df.height:,} rows, {transactions_df.width} columns")
print(f"Customers: {customers_df.height:,} rows, {customers_df.width} columns")
print(f"Articles: {articles_df.height:,} rows, {articles_df.width} columns")

Loading raw data files...
Loading transactions from data\raw\transactions_train.csv ...


FileNotFoundError: File not found: data\raw\transactions_train.csv
Please ensure the dataset is available at the specified path.

In [ ]:
print("Loading raw data files...")

# Robust data loading with existence checks and clear error messages

def _safe_read_csv(path, **kwargs):
    if not Path(path).exists():
        raise FileNotFoundError(f"File not found: {path}\n"
                                "Please ensure the dataset is available at the specified path.")
    return pl.read_csv(path, **kwargs)

# Load transactions data
transactions_path = Path(data_path) / "transactions_train.csv"
print(f"Loading transactions from {transactions_path} ...")
transactions_df = _safe_read_csv(transactions_path, try_parse_dates=True)

# Load customers data
customers_path = Path(data_path) / "customers.csv"
print(f"Loading customers from {customers_path} ...")
customers_df = _safe_read_csv(customers_path, try_parse_dates=True)

# Load articles data
articles_path = Path(data_path) / "articles.csv"
print(f"Loading articles from {articles_path} ...")
articles_df = _safe_read_csv(articles_path, try_parse_dates=True)

print(f"Transactions: {transactions_df.height:,} rows, {transactions_df.width} columns")
print(f"Customers: {customers_df.height:,} rows, {customers_df.width} columns")
print(f"Articles: {articles_df.height:,} rows, {articles_df.width} columns")

In [ ]:
# Display basic info about each dataset
print("=== TRANSACTIONS SCHEMA ===")
print(transactions_df.dtypes)
print("\nFirst few rows:")
print(transactions_df.head(3))

print("\n=== CUSTOMERS SCHEMA ===")
print(customers_df.dtypes)
print("\nFirst few rows:")
print(customers_df.head(3))

print("\n=== ARTICLES SCHEMA ===")
print(articles_df.dtypes)
print("\nFirst few rows:")
print(articles_df.head(3))

## 2. Data Quality Assessment


In [ ]:
def assess_data_quality(df, dataset_name):
    """
    Assess data quality including missing values, duplicates, and basic statistics
    """
    print(f"\n=== DATA QUALITY ASSESSMENT: {dataset_name.upper()} ===")
    
    total_rows = df.height
    print(f"Total rows: {total_rows:,}")
    
    # Check for missing values
    print("\n--- Missing Values ---")
    null_counts = df.null_count()
    
    for column in df.columns:
        null_count = null_counts.select(column).item()
        null_pct = (null_count / total_rows) * 100
        print(f"{column}: {null_count:,} ({null_pct:.2f}%)")
    
    # Check for duplicates
    print("\n--- Duplicate Analysis ---")
    distinct_rows = df.unique().height
    duplicate_count = total_rows - distinct_rows
    print(f"Duplicate rows: {duplicate_count:,} ({(duplicate_count/total_rows)*100:.2f}%)")
    
    return null_counts, duplicate_count

# Assess each dataset
transactions_quality = assess_data_quality(transactions_df, "transactions")
customers_quality = assess_data_quality(customers_df, "customers")
articles_quality = assess_data_quality(articles_df, "articles")

## 3. Duplicate Removal


In [ ]:
def remove_duplicates(df, dataset_name):
    """
    Remove duplicate rows from dataset
    """
    print(f"\n=== REMOVING DUPLICATES: {dataset_name.upper()} ===")
    
    original_count = df.height
    df_deduplicated = df.unique()
    final_count = df_deduplicated.height
    
    removed_count = original_count - final_count
    print(f"Original rows: {original_count:,}")
    print(f"After deduplication: {final_count:,}")
    print(f"Removed duplicates: {removed_count:,} ({(removed_count/original_count)*100:.2f}%)")
    
    return df_deduplicated

# Remove duplicates from all datasets
transactions_clean = remove_duplicates(transactions_df, "transactions")
customers_clean = remove_duplicates(customers_df, "customers")
articles_clean = remove_duplicates(articles_df, "articles")

## 4. Missing Value Handling


In [ ]:
def handle_transactions_missing_values(df):
    """
    Handle missing values in transactions dataset according to user specifications
    """
    print("\n=== HANDLING MISSING VALUES: TRANSACTIONS ===")
    
    original_count = df.height
    
    # 1. Drop rows with missing customer_id or article_id
    df = df.filter(
        (pl.col("customer_id").is_not_null()) & 
        (pl.col("customer_id") != "") &
        (pl.col("article_id").is_not_null()) & 
        (pl.col("article_id") != "")
    )
    
    after_id_drop = df.height
    print(f"Dropped {original_count - after_id_drop:,} rows with missing customer_id or article_id")
    
    # 2. Fill missing prices with median price for that article_id
    # Calculate median price per article_id
    article_median_prices = (
        df.filter(pl.col("price").is_not_null())
        .group_by("article_id")
        .agg(pl.col("price").median().alias("median_price"))
    )
    
    # Join and fill missing prices
    df = df.join(article_median_prices, on="article_id", how="left")
    df = df.with_columns(
        pl.when(pl.col("price").is_null())
        .then(pl.col("median_price"))
        .otherwise(pl.col("price"))
        .alias("price")
    ).drop("median_price")
    
    # 3. Fill missing sales_channel_id with mode
    mode_channel = (
        df.filter(pl.col("sales_channel_id").is_not_null())
        .group_by("sales_channel_id")
        .agg(pl.count().alias("count"))
        .sort("count", descending=True)
        .select("sales_channel_id")
        .item(0, 0)
    )
    
    df = df.with_columns(
        pl.when(pl.col("sales_channel_id").is_null())
        .then(pl.lit(mode_channel))
        .otherwise(pl.col("sales_channel_id"))
        .alias("sales_channel_id")
    )
    
    # 4. Handle missing t_dat (interpolate based on customer patterns)
    # Calculate median date per customer (convert to days since epoch for median calculation)
    customer_median_dates = (
        df.filter(pl.col("t_dat").is_not_null())
        .with_columns(
            pl.col("t_dat").str.to_date("%Y-%m-%d").dt.epoch("d").alias("date_days")
        )
        .group_by("customer_id")
        .agg(pl.col("date_days").median().alias("median_date_days"))
    )
    
    # Join and fill missing dates
    df = df.join(customer_median_dates, on="customer_id", how="left")
    df = df.with_columns(
        pl.when(pl.col("t_dat").is_null())
        .then(
            pl.from_epoch(pl.col("median_date_days") * 24 * 60 * 60, "s")
            .dt.strftime("%Y-%m-%d")
        )
        .otherwise(pl.col("t_dat"))
        .alias("t_dat")
    ).drop("median_date_days")
    
    final_count = df.height
    print(f"Final transaction count: {final_count:,}")
    
    return df

transactions_clean = handle_transactions_missing_values(transactions_clean)

In [ ]:
def handle_customers_missing_values(df):
    """
    Handle missing values in customers dataset according to user specifications
    """
    print("\n=== HANDLING MISSING VALUES: CUSTOMERS ===")
    
    # Calculate median age for imputation
    median_age = df.filter(pl.col("age").is_not_null()).select(pl.col("age").median()).item()
    print(f"Median age for imputation: {median_age}")
    
    # Apply missing value handling rules
    df = df.with_columns([
        # FN: Fill with 0
        pl.when((pl.col("FN").is_null()) | (pl.col("FN") == ""))
        .then(pl.lit(0))
        .otherwise(pl.col("FN"))
        .alias("FN"),
        
        # Active: Fill with "UNKNOWN"
        pl.when((pl.col("Active").is_null()) | (pl.col("Active") == ""))
        .then(pl.lit("UNKNOWN"))
        .otherwise(pl.col("Active"))
        .alias("Active"),
        
        # club_member_status: Fill with "INACTIVE"
        pl.when((pl.col("club_member_status").is_null()) | (pl.col("club_member_status") == ""))
        .then(pl.lit("INACTIVE"))
        .otherwise(pl.col("club_member_status"))
        .alias("club_member_status"),
        
        # fashion_news_frequency: Fill with "NONE"
        pl.when((pl.col("fashion_news_frequency").is_null()) | (pl.col("fashion_news_frequency") == ""))
        .then(pl.lit("NONE"))
        .otherwise(pl.col("fashion_news_frequency"))
        .alias("fashion_news_frequency"),
        
        # age: Fill with median age
        pl.when(pl.col("age").is_null())
        .then(pl.lit(median_age))
        .otherwise(pl.col("age"))
        .alias("age"),
        
        # postal_code: Fill with "UNKNOWN"
        pl.when((pl.col("postal_code").is_null()) | (pl.col("postal_code") == ""))
        .then(pl.lit("UNKNOWN"))
        .otherwise(pl.col("postal_code"))
        .alias("postal_code")
    ])
    
    print("Customer missing values handled successfully")
    return df

customers_clean = handle_customers_missing_values(customers_clean)

In [ ]:
def handle_articles_missing_values(df):
    """
    Handle missing values in articles dataset according to user specifications
    """
    print("\n=== HANDLING MISSING VALUES: ARTICLES ===")
    
    original_count = df.height
    
    # Drop rows with missing article_id
    df = df.filter(
        (pl.col("article_id").is_not_null()) & (pl.col("article_id") != "")
    )
    after_id_drop = df.height
    print(f"Dropped {original_count - after_id_drop:,} rows with missing article_id")
    
    # Handle categorical fields - fill with "UNKNOWN"
    categorical_fields = [
        "product_code", "prod_name", "product_type_name", "product_group_name",
        "graphical_appearance_name", "colour_group_name", "perceived_colour_value_name",
        "perceived_colour_master_name", "department_name", "index_name",
        "index_group_name", "section_name", "garment_group_name", "index_code"
    ]
    
    # Handle numerical code fields - fill with 0
    numerical_fields = [
        "product_type_no", "graphical_appearance_no", "colour_group_code",
        "perceived_colour_value_id", "perceived_colour_master_id", "department_no",
        "index_group_no", "section_no", "garment_group_no"
    ]
    
    # Build expressions for all columns
    expressions = []
    
    for col_name in df.columns:
        if col_name in categorical_fields:
            expressions.append(
                pl.when((pl.col(col_name).is_null()) | (pl.col(col_name) == ""))
                .then(pl.lit("UNKNOWN"))
                .otherwise(pl.col(col_name))
                .alias(col_name)
            )
        elif col_name in numerical_fields:
            expressions.append(
                pl.when(pl.col(col_name).is_null())
                .then(pl.lit(0))
                .otherwise(pl.col(col_name))
                .alias(col_name)
            )
        elif col_name == "detail_desc":
            expressions.append(
                pl.when((pl.col(col_name).is_null()) | (pl.col(col_name) == ""))
                .then(pl.lit("NO_DESCRIPTION"))
                .otherwise(pl.col(col_name))
                .alias(col_name)
            )
        else:
            expressions.append(pl.col(col_name))
    
    df = df.with_columns(expressions)
    
    final_count = df.height
    print(f"Final article count: {final_count:,}")
    print("Article missing values handled successfully")
    
    return df

articles_clean = handle_articles_missing_values(articles_clean)

## 5. Outlier Detection and Handling


In [ ]:
def detect_and_handle_outliers(df, dataset_name, numerical_columns):
    """
    Detect and handle outliers using IQR method with capping
    """
    print(f"\n=== OUTLIER DETECTION AND HANDLING: {dataset_name.upper()} ===")
    
    for col_name in numerical_columns:
        if col_name in df.columns:
            print(f"\nProcessing outliers for {col_name}:")
            
            # Calculate quartiles
            quartiles = df.select([
                pl.col(col_name).quantile(0.25).alias("q1"),
                pl.col(col_name).quantile(0.75).alias("q3")
            ]).to_dict(as_series=False)
            
            q1, q3 = quartiles["q1"][0], quartiles["q3"][0]
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            
            print(f"  Q1: {q1}, Q3: {q3}, IQR: {iqr}")
            print(f"  Lower bound: {lower_bound}, Upper bound: {upper_bound}")
            
            # Count outliers
            outlier_count = df.filter(
                (pl.col(col_name) < lower_bound) | (pl.col(col_name) > upper_bound)
            ).height
            
            print(f"  Outliers detected: {outlier_count:,}")
            
            # Cap outliers instead of removing them
            df = df.with_columns(
                pl.when(pl.col(col_name) < lower_bound)
                .then(pl.lit(lower_bound))
                .when(pl.col(col_name) > upper_bound)
                .then(pl.lit(upper_bound))
                .otherwise(pl.col(col_name))
                .alias(col_name)
            )
            
            print(f"  Outliers capped to bounds")
    
    return df

# Handle outliers for each dataset
transactions_clean = detect_and_handle_outliers(
    transactions_clean, "transactions", ["price"]
)

customers_clean = detect_and_handle_outliers(
    customers_clean, "customers", ["age"]
)

articles_clean = detect_and_handle_outliers(
    articles_clean, "articles", 
    ["product_type_no", "graphical_appearance_no", "colour_group_code",
     "perceived_colour_value_id", "perceived_colour_master_id", "department_no",
     "index_group_no", "section_no", "garment_group_no"]
)

## 6. Data Validation


In [ ]:
def validate_cleaned_data(transactions_df, customers_df, articles_df):
    """
    Perform comprehensive data validation on cleaned datasets
    """
    print("\n=== DATA VALIDATION ===")
    
    validation_results = {}
    
    # 1. Check for remaining null values
    print("\n--- Checking for remaining null values ---")
    datasets = [(transactions_df, "transactions"), (customers_df, "customers"), (articles_df, "articles")]
    
    for df, name in datasets:
        null_counts = df.null_count()
        has_nulls = False
        
        for col_name in df.columns:
            null_count = null_counts.select(col_name).item()
            if null_count > 0:
                if not has_nulls:
                    print(f"{name}: Found nulls in columns:")
                    has_nulls = True
                print(f"  {col_name}: {null_count:,} nulls")
        
        if not has_nulls:
            print(f"{name}: No null values found ✓")
        
        validation_results[f"{name}_has_nulls"] = has_nulls
    
    # 2. Check data consistency
    print("\n--- Data consistency checks ---")
    
    # Check if all customer_ids in transactions exist in customers
    transaction_customers = transactions_df.select("customer_id").unique()
    customer_ids = customers_df.select("customer_id").unique()
    
    missing_customers = transaction_customers.join(
        customer_ids, on="customer_id", how="anti"
    ).height
    print(f"Customers in transactions but not in customers table: {missing_customers:,}")
    validation_results["missing_customers"] = missing_customers
    
    # Check if all article_ids in transactions exist in articles
    transaction_articles = transactions_df.select("article_id").unique()
    article_ids = articles_df.select("article_id").unique()
    
    missing_articles = transaction_articles.join(
        article_ids, on="article_id", how="anti"
    ).height
    print(f"Articles in transactions but not in articles table: {missing_articles:,}")
    validation_results["missing_articles"] = missing_articles
    
    # 3. Check data ranges
    print("\n--- Data range validation ---")
    
    # Age validation
    age_stats = customers_df.select([
        pl.col("age").min().alias("min_age"),
        pl.col("age").max().alias("max_age"),
        pl.col("age").mean().alias("avg_age")
    ]).to_dict(as_series=False)
    
    print(f"Age range: {age_stats['min_age'][0]:.0f} - {age_stats['max_age'][0]:.0f} (avg: {age_stats['avg_age'][0]:.1f})")
    
    # Price validation
    price_stats = transactions_df.select([
        pl.col("price").min().alias("min_price"),
        pl.col("price").max().alias("max_price"),
        pl.col("price").mean().alias("avg_price")
    ]).to_dict(as_series=False)
    
    print(f"Price range: {price_stats['min_price'][0]:.4f} - {price_stats['max_price'][0]:.4f} (avg: {price_stats['avg_price'][0]:.4f})")
    
    # Date validation
    date_stats = transactions_df.select([
        pl.col("t_dat").min().alias("min_date"),
        pl.col("t_dat").max().alias("max_date")
    ]).to_dict(as_series=False)
    
    print(f"Date range: {date_stats['min_date'][0]} - {date_stats['max_date'][0]}")
    
    validation_results["age_stats"] = age_stats
    validation_results["price_stats"] = price_stats
    validation_results["date_stats"] = date_stats
    
    print("\n=== VALIDATION COMPLETE ===")
    return validation_results

validation_results = validate_cleaned_data(transactions_clean, customers_clean, articles_clean)

## 7. Final Data Summary


In [ ]:
def generate_final_summary(transactions_df, customers_df, articles_df):
    """
    Generate final summary of cleaned datasets
    """
    print("\n=== FINAL CLEANED DATA SUMMARY ===")
    
    datasets = [
        (transactions_df, "Transactions"),
        (customers_df, "Customers"), 
        (articles_df, "Articles")
    ]
    
    for df, name in datasets:
        print(f"\n--- {name} ---")
        print(f"Rows: {df.height:,}")
        print(f"Columns: {df.width}")
        print(f"Columns: {', '.join(df.columns)}")
        
        # Show sample data
        print(f"\nSample data:")
        print(df.head(3))

generate_final_summary(transactions_clean, customers_clean, articles_clean)

## 8. Export to Parquet Format


In [ ]:
def save_cleaned_data(transactions_df, customers_df, articles_df, output_path="data/processed/"):
    """
    Save cleaned datasets as Parquet files
    """
    print("\n=== SAVING CLEANED DATA ===")
    
    # Create output directory if it doesn't exist
    Path(output_path).mkdir(parents=True, exist_ok=True)
    
    try:
        # Save transactions
        print("Saving transactions...")
        transactions_df.write_parquet(f"{output_path}transactions_cleaned.parquet")
        print(f"✓ Transactions saved to {output_path}transactions_cleaned.parquet")
        
        # Save customers
        print("Saving customers...")
        customers_df.write_parquet(f"{output_path}customers_cleaned.parquet")
        print(f"✓ Customers saved to {output_path}customers_cleaned.parquet")
        
        # Save articles
        print("Saving articles...")
        articles_df.write_parquet(f"{output_path}articles_cleaned.parquet")
        print(f"✓ Articles saved to {output_path}articles_cleaned.parquet")
        
        print("\n=== ALL DATA SAVED SUCCESSFULLY ===")
        
        # Display file sizes
        print("\n--- File Sizes ---")
        for filename in ["transactions_cleaned.parquet", "customers_cleaned.parquet", "articles_cleaned.parquet"]:
            filepath = Path(output_path) / filename
            if filepath.exists():
                size_mb = filepath.stat().st_size / (1024 * 1024)
                print(f"{filename}: {size_mb:.2f} MB")
        
    except Exception as e:
        print(f"Error saving data: {str(e)}")
        raise

save_cleaned_data(transactions_clean, customers_clean, articles_clean)

## 9. Performance Metrics and Summary


In [ ]:
def display_performance_metrics():
    """
    Display final performance metrics and summary
    """
    print("\n=== PERFORMANCE METRICS ===")
    
    print(f"Polars version: {pl.__version__}")
    print(f"Streaming chunk size: {pl.Config.get_streaming_chunk_size()}")
    
    print("\n=== PREPROCESSING COMPLETE ===")
    print("All datasets have been successfully:")
    print("✓ Loaded with Polars for high performance")
    print("✓ Deduplicated")
    print("✓ Missing values handled per specifications")
    print("✓ Outliers processed using IQR capping")
    print("✓ Data validated for consistency")
    print("✓ Saved as optimized Parquet files")
    
    print("\n=== NEXT STEPS ===")
    print("Your cleaned datasets are ready for:")
    print("• Feature engineering")
    print("• Customer segmentation analysis")
    print("• Recommendation system development")
    print("• Statistical modeling and machine learning")

display_performance_metrics()